In [1]:
import torch
from knn_loss import KNNLoss
import utility_functions

In [ ]:
def regen_rand():
    # TODO
    pass

In [2]:
tensor_a = torch.tensor([[2.0], [3.0], [4.0]])
tensor_b = torch.tensor([[0.0], [0.0], [0.0]])
tensor_c = torch.tensor([[-2.0], [-3.0], [-4.0]])

In [3]:
tensors = [tensor_a, tensor_b, tensor_c]
stack = torch.stack(tensors)
torch.mean(stack, dim = 0)
# stack

tensor([[0.],
        [0.],
        [0.]])

In [2]:
forward_pass = torch.randn(256, 128, 1)
labels = torch.rand(256, 1, 1)
labels *= 9
labels = torch.absolute(labels)
labels = torch.round(labels)

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
classes_tensor = torch.tensor(classes)
classes_tensor.shape

torch.Size([10])

In [3]:
wrap = KNNLoss(classes_tensor)

In [6]:
# wrap = KNNLoss(classes_tensor)
# div_loss = wrap.divergence_loss(forward_pass, labels)
# print(div_loss)
# centroids = wrap.get_centroids()
# print(centroids.shape)
# conv_loss = wrap.convergence_loss(forward_pass, labels)
# print(conv_loss)


the `# Calculate Current Centroids` section seem to work.

In [7]:
# print(forward_pass[0])
# print(forward_pass[1])

# a = forward_pass[0]
# b = forward_pass[1]

# cos = torch.nn.CosineSimilarity(dim = 0)
# wrap.cosine_similarity(a, b)

In [4]:
print(wrap.get_centroids())

None


In [6]:
forward_pass = utility_functions.to_device(torch.rand(256, 128, requires_grad=True), utility_functions.get_default_device())
labels = utility_functions.to_device(torch.rand(256, 1, requires_grad=True), utility_functions.get_default_device()).mul(9).absolute().round()

In [7]:
print(labels.max())

tensor(9., device='cuda:0', grad_fn=<MaxBackward1>)


In [8]:
debug_1 = utility_functions.to_device(torch.randn(len(classes_tensor), forward_pass.shape[1]), wrap.device)
print(debug_1)
wrap.centroids = debug_1

tensor([[-0.0661, -1.9436,  0.2760,  ...,  0.0201, -0.2203,  0.6937],
        [-0.9049, -0.5431,  2.0047,  ...,  0.9057, -0.1693, -1.0594],
        [-1.0525, -0.5521,  1.3504,  ...,  0.4818,  0.2720, -1.7054],
        ...,
        [-0.7704,  0.7372, -0.0790,  ..., -0.8952, -0.4362, -2.6234],
        [ 0.6190,  1.7805, -0.4956,  ...,  0.6191, -1.9304,  2.5194],
        [ 0.7160,  0.0582,  0.0110,  ...,  0.5262, -0.5016, -0.1412]],
       device='cuda:0')


In [9]:
c = []
for _c in wrap.classes:
    class_l = [wrap.centroids[_c].detach()]
    for inp, tar in zip(forward_pass, labels):
        if int(tar) == _c:
            class_l.append(inp)
    stacked_t = torch.stack(class_l)
    c.append(torch.mean(stacked_t, dim = 0))
wrap.centroids = torch.stack(c)
print(wrap.centroids)

tensor([[0.3960, 0.3989, 0.3826,  ..., 0.5575, 0.5347, 0.6373],
        [0.5356, 0.4312, 0.5038,  ..., 0.4464, 0.5068, 0.3853],
        [0.4869, 0.4462, 0.5584,  ..., 0.5714, 0.3509, 0.3909],
        ...,
        [0.5254, 0.4863, 0.3490,  ..., 0.5248, 0.4165, 0.3545],
        [0.4607, 0.5573, 0.4094,  ..., 0.4609, 0.4275, 0.5645],
        [0.7235, 0.3993, 0.4834,  ..., 0.6116, 0.4802, 0.5535]],
       device='cuda:0', grad_fn=<StackBackward0>)


In [10]:
d = []
for centroid_a in wrap.centroids:
    local_d = []    # for each centroid collect distances to other centroids in a list. This list should be ALWAYS be 9 elements long
    for centroid_b in wrap.centroids:
        if not torch.equal(centroid_a, centroid_b):
            local_d.append(wrap.cosine_similarity(centroid_a, centroid_b))
    local_t = torch.stack(local_d)
    d.append(torch.mean(local_t))
wrap.mean_centroid_distance = torch.stack(d).abs().mean()
print(wrap.mean_centroid_distance)

tensor(0.9758, device='cuda:0', grad_fn=<MeanBackward0>)


In [11]:
instance_d = []
for inp, tar in zip(forward_pass, labels):
    label = int(tar)
    corresponding_centroid = wrap.centroids[label].detach()
    instance_d.append(wrap.euclidean_distance(inp, corresponding_centroid))
d = torch.mul(torch.stack(instance_d), 1)

In [12]:
convergence = torch.mean(d)
uniformity = torch.std(d)
divergence = torch.div(1, torch.sub(1, wrap.mean_centroid_distance))

In [13]:
print(
    convergence,
    uniformity,
    divergence
)

tensor(3.2372, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1469, device='cuda:0', grad_fn=<StdBackward0>) tensor(41.2923, device='cuda:0', grad_fn=<DivBackward0>)


> _Now the Ratio doesn't make sense anymore..._

In [14]:
print(torch.mul(divergence, convergence.add(uniformity)))

tensor(139.7388, device='cuda:0', grad_fn=<MulBackward0>)


In [17]:
inputs = [utility_functions.to_device(torch.rand(256, 128, requires_grad=True), utility_functions.get_default_device()) for i in range(195)]
targets = [utility_functions.to_device(torch.rand(256, 1, requires_grad=True), utility_functions.get_default_device()).mul(9).absolute().round() for i in range(195)]

loss = KNNLoss(classes_tensor)

In [15]:
inputs = [utility_functions.to_device(torch.rand(256, 128, requires_grad=True), utility_functions.get_default_device()) for i in range(64)]
targets = [utility_functions.to_device(torch.rand(256, 1, requires_grad=True), utility_functions.get_default_device()).mul(9).absolute().round() for i in range(64)]

loss = KNNLoss(classes_tensor)

In [18]:
for input, target in zip(inputs, targets):
    print(
        loss.loss_renewed(input, target)
    )

tensor(150.8646, device='cuda:0', grad_fn=<MulBackward0>)
tensor(258.3460, device='cuda:0', grad_fn=<MulBackward0>)
tensor(266.7856, device='cuda:0', grad_fn=<MulBackward0>)
tensor(251.2249, device='cuda:0', grad_fn=<MulBackward0>)
tensor(262.8551, device='cuda:0', grad_fn=<MulBackward0>)
tensor(251.3738, device='cuda:0', grad_fn=<MulBackward0>)
tensor(211.2651, device='cuda:0', grad_fn=<MulBackward0>)
tensor(268.9648, device='cuda:0', grad_fn=<MulBackward0>)
tensor(271.2145, device='cuda:0', grad_fn=<MulBackward0>)
tensor(257.2696, device='cuda:0', grad_fn=<MulBackward0>)
tensor(241.6110, device='cuda:0', grad_fn=<MulBackward0>)
tensor(233.6987, device='cuda:0', grad_fn=<MulBackward0>)
tensor(270.2541, device='cuda:0', grad_fn=<MulBackward0>)
tensor(240.5599, device='cuda:0', grad_fn=<MulBackward0>)
tensor(271.3982, device='cuda:0', grad_fn=<MulBackward0>)
tensor(242.3234, device='cuda:0', grad_fn=<MulBackward0>)
tensor(251.6206, device='cuda:0', grad_fn=<MulBackward0>)
tensor(234.352